In [1]:
import pandas as pd
import numpy as np
import os
import glob
import cv2
import PIL
from PIL import Image

%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
df_2dFFT = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Blood_Viscosity-2dFFTdataset-3channels-3Fold-EMClasses.csv")
#/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Blood_Viscosity-2dFFTdataset-3channels-3Fold-EMClasses.csv
print(df_2dFFT.shape)
df_2dFFT.head()

(168, 16)


,Unnamed: 0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0.1.1,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM,fold,typeBEvsBM_binary
0,0,0,185,185,HN29_D0_30HZ_20XINF_UWELL_25660117_150029_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN29_D0_30HZ_20XINF_UWELL_25660117_150029,SSD_Backup,HN,HN29,No_Splenectomy,"[2, 5, 7]",1,E,1,0
1,1,1,278,278,HN39_D0_30HZ_20XINF_UWELL_25660223_151435_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN39_D0_30HZ_20XINF_UWELL_25660223_151435,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1,0
2,2,2,280,280,HN39_D0_30HZ_20XINF_UWELL_25660223_151842_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN39_D0_30HZ_20XINF_UWELL_25660223_151842,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1,0
3,3,3,286,286,HN39_D0_30HZ_20XINF_UWELL_25660223_153102_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN39_D0_30HZ_20XINF_UWELL_25660223_153102,SSD_Backup,HN,HN39,Splenectomy,"[2, 5, 7]",1,E,1,0
4,4,4,48,48,HN13_D0_30HZ_20XINF_UWELL_25651124_144313_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN13_D0_30HZ_20XINF_UWELL_25651124_144313,SSD_Backup,HN,HN13,Splenectomy,"[2, 5, 7]",1,E,1,0


In [5]:
subclass = list(set(df_2dFFT["subclass"]))
print(len(subclass))
subclass

for sub_i in subclass:
    print(f"[INFO]: ----- {sub_i} -----")
    df_sub = df_2dFFT[df_2dFFT["subclass"]==sub_i].reset_index(drop=True)
    set_case = list(set(df_sub["Code"]))
    print(f"[INFO]: {df_sub.shape[0]} images ==> {len(set_case)} Cases")
    print("*"*125)

2
[INFO]: ----- No_Splenectomy -----
[INFO]: 65 images ==> 16 Cases
*****************************************************************************************************************************
[INFO]: ----- Splenectomy -----
[INFO]: 103 images ==> 17 Cases
*****************************************************************************************************************************


In [22]:
df_l_len = []
list_code =  list(set(df_2dFFT["Code"]))
print(len(list_code))
#list_code
for l in range(len(list_code)):
    df_l = df_2dFFT[df_2dFFT["Code"]==list_code[l]].reset_index(drop=True)
    print(f"[INFO]: Case: {list_code[l]} ==> len {df_l.shape[0]} images")
    df_l_len.append(df_l.shape[0])

33
[INFO]: Case: HN02 ==> len 4 images
[INFO]: Case: HN23 ==> len 4 images
[INFO]: Case: HN01 ==> len 5 images
[INFO]: Case: HN36 ==> len 12 images
[INFO]: Case: HN16 ==> len 4 images
[INFO]: Case: HN29 ==> len 3 images
[INFO]: Case: HN28 ==> len 8 images
[INFO]: Case: HN14 ==> len 2 images
[INFO]: Case: HN30 ==> len 2 images
[INFO]: Case: HN32 ==> len 2 images
[INFO]: Case: HN21 ==> len 1 images
[INFO]: Case: HN19 ==> len 8 images
[INFO]: Case: HN35 ==> len 3 images
[INFO]: Case: HN22 ==> len 10 images
[INFO]: Case: HN27 ==> len 3 images
[INFO]: Case: HN24 ==> len 8 images
[INFO]: Case: HN39 ==> len 6 images
[INFO]: Case: HN31 ==> len 2 images
[INFO]: Case: HN44 ==> len 1 images
[INFO]: Case: HN25 ==> len 8 images
[INFO]: Case: HN33 ==> len 2 images
[INFO]: Case: HN40 ==> len 12 images
[INFO]: Case: HN17 ==> len 6 images
[INFO]: Case: HN12 ==> len 7 images
[INFO]: Case: HN42 ==> len 2 images
[INFO]: Case: HN38 ==> len 5 images
[INFO]: Case: HN13 ==> len 5 images
[INFO]: Case: HN41 ==>

In [24]:
print("Min:" ,min(df_l_len))
print("Max:", max(df_l_len)) 

Min 1
Max 12


In [ ]:
#grouped = .groupby(["A", "B"])

In [12]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    lst_fold.sort()
    for j in range(len(lst_fold)):
        df_classes = DataSet[DataSet["fold"]==lst_fold[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        lst_folder = list(set(df_classes["fold"]))
        print(f"[INFO]: [{j+1}].  Fold {lst_fold[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 6-Fold With Class:")
        lst_cls = list(set(df_classes['typeBEvsBM']))
        print(f"Number of Classes {len(lst_cls)} Class ==> {lst_cls}")
        for m in lst_cls:
            df_f =  df_classes[df_classes["typeBEvsBM"]==m]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]:  Classse {m} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*100)
    
    
    return

In [13]:
_(df_2dFFT)

[INFO]: [1].  Fold 1 With Shape ==> 56 Folder, (56, 16)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 28 Folder, 28 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 56 Folder, (56, 16)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 28 Folder, 28 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 56 Folder, (56, 16)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['E', 'M']
[INFO]:  Classse E ==> With 28 Folder, 28 images!! ==> 8 PerSon!!
[INFO]:  Classse M ==> With 28 Folder, 28 images!! ==> 3 PerSon!!


------------------

In [25]:
dt_2 = 0.045344
dt_5 = 0.122015
dt_7 = 0.164181

In [26]:
dt_7-dt_5

0.042165999999999995

In [27]:
dt_5-dt_2

0.07667099999999999